# Import lib and master data

In [ ]:
#Upload the "MasterData.xlsx" excel

In [ ]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import random
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

In [ ]:
# install faker for generating fake customer names
!pip install faker

In [ ]:
#Install Pytorch geometirc to generate hetero graph data
!pip install torch-geometric -q

In [ ]:
from faker import Faker
faker = Faker()

In [ ]:
faker_us = Faker('en_US')       # American
faker_uk = Faker('en_GB')       # British
faker_aus = Faker('en_AU')      # Australian
faker_ind = Faker('en_IN')      # Indian
Faker.seed(42)

In [ ]:
# Set the path to the master data file and conditions file
master_data_path = "MasterData.xlsx"

# Load all sheets in both the files
master_data = pd.read_excel(master_data_path, sheet_name=None)

# Print the sheet names
print("Master Data Sheets:", list(master_data.keys()))


Master Data Sheets: ['CustSegment', 'CustCity', 'CustRisk', 'CustEmpStatus', 'CustIncomeGroup', 'CustAgeGroup', 'CustGender', 'CustMaritalStatus', 'CustEducation', 'CustCreditScore', 'ProdType', 'ProdSubType', 'ProdRisk', 'ProdMinBal', 'ProdTenor', 'ProdFee', 'InterType', 'InterChannel', 'InterAmt', 'InterFreq']


In [ ]:
#Extract each masterdata excel sheet into a separate dataframe
custseg_master = master_data["CustSegment"]
custcity_master = master_data["CustCity"]
custrisk_master = master_data["CustRisk"]
custempstatus_master = master_data["CustEmpStatus"]
custinc_master = master_data["CustIncomeGroup"]
custage_master = master_data["CustAgeGroup"]
custgender_master = master_data["CustGender"]
custmarital_master = master_data['CustMaritalStatus']
custedu_master = master_data['CustEducation']
custcredit_master = master_data['CustCreditScore']
prodtype_master = master_data['ProdType']
prodsubtype_master = master_data['ProdSubType']
prodrisk_master = master_data['ProdRisk']
prodminbal_master = master_data['ProdMinBal']
prodtenor_master = master_data['ProdTenor']
prodfee_master = master_data['ProdFee']

# Customer data

In [ ]:
#Generate Customer data
#Weighted sampling across locales
def generate_mixed_name():
    locale_choice = random.choices(
        population=['us', 'uk', 'aus', 'ind'],
        weights=[0.3, 0.2, 0.2, 0.3],  # Adjust weights as needed
        k=1
    )[0]

    if locale_choice == 'us':
        return faker_us.name()
    elif locale_choice == 'uk':
        return faker_uk.name()
    elif locale_choice == 'aus':
        return faker_aus.name()
    else:
        return faker_ind.name()

In [ ]:
custseg_master["CustSegName"] = custseg_master["CustSegName"].str.strip()

In [ ]:
custgender_master["CustGender"] = custgender_master["CustGender"].str.strip()

In [ ]:
# Define age range and probabilities
age_group_ranges = {
    "Young Adult / Student / Early Professional": (18, 24),
    "Early Career": (25, 34),
    "Mid Career / Family Builders": (35, 44),
    "Senior Professional / SME Owner": (45, 54),
    "Pre-Retirement": (55, 64),
    "Senior Citizen / Retired": (65, 80)   # you can cap at 80 or 90
}

age_group_probs = {
    "18-24": 0.12,
    "25-34": 0.28,
    "35-44": 0.25,
    "45-54": 0.15,
    "55-64": 0.10,
    "65+":   0.10
}

In [ ]:
#Define income range and probabilities

inc_group_ranges = {
    "Low Income / Entry level": (0,250000),
    "Lower Middle Income": (251000,500000),
    "Mass Market Salaried / SME owners": (500001,1000000),
    "Mass Affluent / Upper Mid": (1000001,2000000),
    "Affluent": (2000001,5000000),
    "HNI": (5000001,10000000),
    "Ultra-HNI / Private Banking":(10000001,100000000)}

income_bracket_probs = {
 "<2.5L": 0.15,
 "2.5L–5L": 0.20,
 "5L–10L": 0.25,
 "10L–20L": 0.20,
 "20L–50L": 0.10,
 "50L–1Cr": 0.05,
 ">1Cr": 0.03
}

In [ ]:
#Define customer credit score range and probabilities

cscore_range = {
    "Poor" : (300,549),
    "Below Average": (550, 649),
    "Fair":(650,699),
    "Good": (700,749),
    "Very Good": (750,799),
    "Excellent": (800,900)}

cscore_probs = {
 "300-549": 0.05,
 "550-649": 0.10,
 "650-699": 0.15,
 "700-749": 0.25,
 "750-799": 0.25,
 "800-900": 0.15
}

In [ ]:
#Define a function to ensure the synthetic customer generation has a realistic mix, set thresholds for the same

segment_probs = {
 "General Retail": 0.34,
 "Digital-First Customer": 0.15,
 "Mass Affluent": 0.10,
 "Corporate Employee (Salary Accounts)": 0.08,
 "SME / Business Owner": 0.06,
 "Student / Young Professional": 0.05,
 "Senior Citizen": 0.06,
 "Self-Employed Professional": 0.04,
 "Government / PSU Employee": 0.03,
 "NRI (Non-Resident Indian)": 0.03,
 "Priority / Imperia Banking": 0.02,
 "HNWI (High Net Worth Individual)": 0.015,
 "Ultra-HNI / Private Banking": 0.005,
 "Rural Banking / Agri Customer": 0.01,
 "Branch-Banking Loyalist": 0.01
}

city_tier_probs = {
 "Metro": 0.30,
 "Tier-1": 0.25,
 "Tier-2": 0.20,
 "Tier-3": 0.15,
 "Rural": 0.10
}

customer_risk_probs = {
 "Very Low": 0.15,
 "Low": 0.30,
 "Medium": 0.34,
 "High": 0.15,
 "Very High": 0.05,
 "WatchList": 0.01
}

employment_status_probs = {
 "Salaried – Private Sector": 0.40,
 "Salaried – Government/PSU": 0.10,
 "Self-Employed – Business Owner / SME": 0.15,
 "Self-Employed – Professional": 0.07,
 "Student": 0.05,
 "Homemaker / Dependent": 0.05,
 "Unemployed / Job Seeker": 0.03,
 "Retired / Pensioner": 0.07,
 "NRI / Overseas Worker": 0.05
}

gender_probs = {
 "Male": 0.60,
 "Female": 0.39,
 "Other": 0.01
}

marital_status_probs = {
 "Single / Never Married": 0.35,
 "Married": 0.55,
 "Divorced / Separated": 0.03,
 "Widowed": 0.02,
 "Prefer Not to Say / Other": 0.05
}

education_probs = {
 "No Formal Education / Primary": 0.05,
 "Secondary School (Up to 10th Std)": 0.10,
 "Higher Secondary (11th–12th Std)": 0.15,
 "Diploma / Polytechnic": 0.10,
 "Undergraduate (Bachelor’s Degree)": 0.30,
 "Postgraduate (Master’s, MBA, MCA, etc.)": 0.20,
 "Doctorate / Professional (CA, CS, PhD, Medicine, Law, etc.)": 0.05,
 "Prefer Not to Say / Not Captured": 0.05
}

In [ ]:
#Define Functions to sample different features
num_customers = 100000

#def sample_age_from_group(group):
    #low, high = age_group_ranges[group]
    #return np.random.randint(low, high+1)  # inclusive

def sample_age_from_group(group):
    low, high = age_group_ranges[group]
    mean = (low + high) / 2
    std = (high - low) / 4  # ~95% within group
    age = int(np.clip(np.random.normal(mean, std), low, high))
    return age

def sample_inc_from_group(group):
    low, high = inc_group_ranges[group]
    mean = (low + high) / 2
    std = (high - low) / 4  # ~95% within group
    income = int(np.clip(np.random.normal(mean, std), low, high))
    return income

def sample_cscore_from_group(group):
    low, high = cscore_range[group]
    mean = (low + high) / 2
    std = (high - low) / 4  # ~95% within group
    cscore = int(np.clip(np.random.normal(mean, std), low, high))
    return cscore

def sample_with_prob(master_df, col_name, col_code, target_dist, total_needed):
    samples = []

    for prob, fraction in target_dist.items():
        subset = master_df[master_df[col_name] == prob]
        n_samples = int(total_needed * fraction)
        sampled = subset.sample(n=n_samples, replace=(n_samples > len(subset)))
        samples.append(sampled[[col_code]])

    return pd.concat(samples, ignore_index=True)


# Samples
segment_sample = sample_with_prob(custseg_master, "CustSegName", "CustSegName", segment_probs, num_customers)
city_sample = sample_with_prob(custcity_master, "Tier", "CustCity", city_tier_probs, num_customers)
risk_sample = sample_with_prob(custrisk_master, "CustRisk", "CustRisk", customer_risk_probs, num_customers)
empstat_sample = sample_with_prob(custempstatus_master, "CustEmpStatus", "CustEmpStatus", employment_status_probs, num_customers)
income_sample = sample_with_prob(custinc_master, "Income", "CustIncomeGroup", income_bracket_probs, num_customers)
age_group_sample = sample_with_prob(custage_master, "Age", "CustAgeGroup", age_group_probs, num_customers)
gender_sample = sample_with_prob(custgender_master, "CustGender", "CustGender", gender_probs, num_customers)
marital_sample = sample_with_prob(custmarital_master, "CustMaritalStatus", "CustMaritalStatus", marital_status_probs, num_customers)
edu_sample = sample_with_prob(custedu_master, "CustEducation", "CustEducation", education_probs, num_customers)
cscore_sample = sample_with_prob(custcredit_master, "CreditScore", "CustCreditScore", cscore_probs, num_customers)



In [ ]:
# Generate 100000 customers
start_date = pd.to_datetime("2023-01-01")
end_date = pd.to_datetime("2025-09-15")
date_range = pd.date_range(start=start_date, end=end_date).to_list()

customers = []
for i in range(100000):
    #cust_type = cust_type_sample.iloc[i % len(cust_type_sample)]["CustomerTypeCode"]
    name = generate_mixed_name()
    cust = {
        "CustomerID": f"CUST{i+1:06}",
        "CustomerName": name,
        "CustomerAge" : sample_age_from_group(age_group_sample.iloc[i % len(age_group_sample)]["CustAgeGroup"]),
        "CustomerIncome": sample_inc_from_group(income_sample.iloc[i % len(income_sample)]["CustIncomeGroup"]),
        #"CustomerIncome" : income_sample.iloc[i % len(income_sample)]["CustIncomeGroup"],
        "CustomerSegment": segment_sample.iloc[i % len(segment_sample)]["CustSegName"],
        "CustomerCity" : city_sample.iloc[i % len(city_sample)]["CustCity"],
        "CustomerRisk" : risk_sample.iloc[i % len(risk_sample)]["CustRisk"],
        "CustomerEmployment": empstat_sample.iloc[i % len(empstat_sample)]["CustEmpStatus"],
        "CustomerGender" : gender_sample.iloc[i % len(gender_sample)]["CustGender"],
        "CustomerMaritalStatus": marital_sample.iloc[i % len(marital_sample)]["CustMaritalStatus"],
        "CustomerEducation" : edu_sample.iloc[i % len(edu_sample)]["CustEducation"],
        #"CustomerCreditScore": cscore_sample.iloc[i % len(cscore_sample)]["CustCreditScore"],
        "CustomerCreditScore": sample_cscore_from_group(cscore_sample.iloc[i % len(cscore_sample)]["CustCreditScore"]),
        "CustomerCreatedDate": random.choice(date_range).strftime("%Y-%m-%d"),

    }
    customers.append(cust)


df_customers = pd.DataFrame(customers)

In [ ]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   CustomerID             100000 non-null  object
 1   CustomerName           100000 non-null  object
 2   CustomerAge            100000 non-null  int64 
 3   CustomerIncome         100000 non-null  int64 
 4   CustomerSegment        100000 non-null  object
 5   CustomerCity           100000 non-null  object
 6   CustomerRisk           100000 non-null  object
 7   CustomerEmployment     100000 non-null  object
 8   CustomerGender         100000 non-null  object
 9   CustomerMaritalStatus  100000 non-null  object
 10  CustomerEducation      100000 non-null  object
 11  CustomerCreditScore    100000 non-null  int64 
 12  CustomerCreatedDate    100000 non-null  object
dtypes: int64(3), object(10)
memory usage: 9.9+ MB


In [ ]:
df_customers.head()

,CustomerID,CustomerName,CustomerAge,CustomerIncome,CustomerSegment,CustomerCity,CustomerRisk,CustomerEmployment,CustomerGender,CustomerMaritalStatus,CustomerEducation,CustomerCreditScore,CustomerCreatedDate
0,CUST000001,Allison Hill,20,128799,General Retail,Kolkata,Very Low,Salaried – Private Sector,Male,Single / Never Married,No Formal Education / Primary,342,2024-04-17
1,CUST000002,Noah Rhodes,18,87526,General Retail,Kolkata,Very Low,Salaried – Private Sector,Male,Single / Never Married,No Formal Education / Primary,413,2023-04-01
2,CUST000003,Angie Henderson,20,85668,General Retail,Delhi NCR,Very Low,Salaried – Private Sector,Male,Single / Never Married,No Formal Education / Primary,372,2025-02-21
3,CUST000004,Joanne Lloyd,22,153495,General Retail,Delhi NCR,Very Low,Salaried – Private Sector,Male,Single / Never Married,No Formal Education / Primary,367,2024-07-03
4,CUST000005,Alyssa Gonzalez,20,226720,General Retail,Chennai,Very Low,Salaried – Private Sector,Male,Single / Never Married,No Formal Education / Primary,435,2025-08-08


In [ ]:
df_customers["CustomerSegment"].value_counts()

,count
CustomerSegment,
General Retail,34000
Digital-First Customer,15000
Mass Affluent,10000
Corporate Employee (Salary Accounts),8000
SME / Business Owner,6000
Senior Citizen,6000
Student / Young Professional,5000
Self-Employed Professional,4000
Government / PSU Employee,3000


In [ ]:
df_customers["CustomerCity"].value_counts()

,count
CustomerCity,
Chennai,27
Hyderabad,22
Mumbai,21
Bengaluru,18
Delhi NCR,16
...,...
Ramanagara (Karnataka),1
Palakkad (Kerala),1
Sivakasi (Tamil Nadu),1


In [ ]:
df_customers["CustomerRisk"].value_counts()

,count
CustomerRisk,
Medium,136
Low,120
Very Low,60
High,60
Very High,20
WatchList,4


In [ ]:
df_customers["CustomerEmployment"].value_counts()

,count
CustomerEmployment,
Salaried – Private Sector,172
Self-Employed – Business Owner / SME,60
Salaried – Government/PSU,40
Self-Employed – Professional,28
Retired / Pensioner,28
Student,20
Homemaker / Dependent,20
NRI / Overseas Worker,20
Unemployed / Job Seeker,12


In [ ]:
df_customers["CustomerGender"].value_counts()

,count
CustomerGender,
Male,240
Female,156
Other,4


In [ ]:
df_customers["CustomerMaritalStatus"].value_counts()

,count
CustomerMaritalStatus,
Married,220
Single / Never Married,140
Prefer Not to Say / Other,20
Divorced / Separated,12
Widowed,8


In [ ]:
df_customers["CustomerEducation"].value_counts()

,count
CustomerEducation,
Undergraduate (Bachelor’s Degree),120
"Postgraduate (Master’s, MBA, MCA, etc.)",80
Higher Secondary (11th–12th Std),60
Secondary School (Up to 10th Std),40
Diploma / Polytechnic,40
No Formal Education / Primary,20
"Doctorate / Professional (CA, CS, PhD, Medicine, Law, etc.)",20
Prefer Not to Say / Not Captured,20


In [ ]:
df_customers["CustomerCreditScore"].value_counts()

,count
CustomerCreditScore,
727,8
767,7
730,6
770,5
716,5
...,...
403,1
396,1
483,1


In [ ]:
df_customers["CustomerCreditScore"].min(), df_customers["CustomerCreditScore"].max()

(300, 900)

In [ ]:
df_customers["CustomerAge"].min(), df_customers["CustomerAge"].max()

(18, 80)

In [ ]:
df_customers["CustomerIncome"].min(), df_customers["CustomerIncome"].max()

(0, 100000000)

# Product data

In [ ]:
# Generate synthitic product data (42 products + 1 dummy product)
# Select only the columns you want from each master
risk_df     = prodrisk_master[["ProdSubType", "ProdRisk"]]
minbal_df   = prodminbal_master[["ProdSubType", "ProdMinBal","MinBalCat"]]
tenor_df    = prodtenor_master[["ProdSubType", "ProdTenor","ProdTenorCat"]]
fee_df      = prodfee_master[["ProdSubType", "ProdFee","ProdFeeCat"]]

# Start with prodsubtype_master and merge one by one
product_master_df = prodsubtype_master.copy()

product_master_df = product_master_df.merge(risk_df, on="ProdSubType", how="left")
product_master_df = product_master_df.merge(minbal_df, on="ProdSubType", how="left")
product_master_df = product_master_df.merge(tenor_df, on="ProdSubType", how="left")
product_master_df = product_master_df.merge(fee_df, on="ProdSubType", how="left")


In [ ]:
product_master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ProdID        43 non-null     object 
 1   ProdType      43 non-null     object 
 2   ProdSubType   43 non-null     object 
 3   Percent       43 non-null     float64
 4   ProdRisk      43 non-null     object 
 5   ProdMinBal    43 non-null     object 
 6   MinBalCat     43 non-null     object 
 7   ProdTenor     43 non-null     object 
 8   ProdTenorCat  43 non-null     object 
 9   ProdFee       43 non-null     object 
 10  ProdFeeCat    43 non-null     object 
dtypes: float64(1), object(10)
memory usage: 3.8+ KB


In [ ]:
product_master_df.head()

,ProdID,ProdType,ProdSubType,Percent,ProdRisk,ProdMinBal,MinBalCat,ProdTenor,ProdTenorCat,ProdFee,ProdFeeCat
0,PST0011,Deposits,Deposits - Savings Account,0.1000,Low,0-10000,Low,On-demand,No fixed tenure,100-500,Low Fee
1,PST0012,Deposits,Deposits - Current Account,0.0250,Low,25000 - 100000,Medium,On-demand,No fixed tenure,100-500,Low Fee
2,PST0013,Deposits,Deposits - Fixed Deposit (FD),0.0750,Low,1000-10000,Low,7 days – 10 years,Medium-term,0,No Fee
3,PST0014,Deposits,Deposits - Recurring Deposit (RD),0.0375,Low,500-1000,Very Low,6 months – 10 years,Medium-term,0,No Fee
4,PST0015,Deposits,Deposits - Salary Account (special SA variant),0.0125,Low,0,Zero / No Minimum,On-demand,No fixed tenure,0,No Fee


In [ ]:
#Convert ProdMinBal column to a numeric column using mid-points
min_bal_mapping = {
    '0-10000': 5000,
    '25000 - 100000': 50000,
    '1000-10000': 5000,
    '500-1000': 750,
    0: 0,
    '25000-50000': 38000,
    '200000+': 200000,
    '50000+': 50000,
    '10000+': 10000,
    '25000+': 25000,
    '100000+': 100000,
    '2000-3000': 2500,
    10000: 10000,
    '2000-5000': 3500,
    '1000+': 1000,
    '500+': 500,
    '1000-2000': 1500,
    5000: 5000,
    300: 300,
    '10000-100000': 50000,
    '25L+': 2500000,
    500: 500,
    '25000-100000': 75000,
    100: 100,
    '10L+': 1000000,
    250: 250,
    42: 1000,
    1000: 1000,
    '1.5L': 150000
}

# Step 2: Apply mapping to the column
product_master_df["ProdMinBalNumeric"] = product_master_df["ProdMinBal"].map(min_bal_mapping)


In [ ]:
#Convert ProdFee column to a numeric column using mid-points
prod_fee_mapping = {
    '100-500': 300,
    0: 0,
    '1000-5000': 3000,
    '5000-25000': 15000,
    '2500-5000': 3750,
    '5000-15000': 10000,
    '500-2000': 1250,
    '1000-3000': 2000,
    '1000-25000': 13000,
    '500-750': 650,
    '2000-5000': 3500,
    '10000-15000': 13000,
    '300-1000': 650,
    '150-750': 450,
    '5000-1L': 50000,
    '500-5000': 2750,
    '300-800': 650,
    '25000-2L': 100000,
    '250-1500': 900,
    '0-10000': 5000
}

# Step 2: Apply mapping to the column
product_master_df["ProdFeeNumeric"] = product_master_df["ProdFee"].map(prod_fee_mapping)


In [ ]:
#Convert ProdTenor column to a numeric column using mid-points
prod_tenor_mapping = {
    'On-demand': 0,
    '7 days – 10 years': 60,
    '6 months – 10 years': 63,
    '1 – 5 years': 36,
    '10 – 30 years': 240,
    '3 – 7 years': 60,
    '5 – 15 years': 120,
    '6 months – 3 years': 21,
    '2 – 5 years': 42,
    '5 – 20 years': 150,
    '3 – 5 years': 48,
    'On-going': 0,
    '10 – 25 years': 210,
    '1 year': 12,
    '1 – 3 years': 24,
    'Per trip': 0,
    '3–15 years': 108,
    '2-7 years': 54,
    '5-7 years': 72,
    '5-15 years': 120,
    '5 years': 60,
    '1-10 years': 66,
    'Shortterm': 0,
    'On-going ': 0,
    '15 years': 180,
    '21 years': 252,
    '8 years': 96,
    '1-15 years': 96,
    '1-5 years': 36
}
product_master_df["ProdTenorNumeric"] = product_master_df["ProdTenor"].map(prod_tenor_mapping)


#Accounts data

In [ ]:
#Generate synthetic accounts data
# Define the ranges and probabilities needed for different account features
acct_bal_range = {
    "0-5000": (500, 5000),
    "5000-25000": (5000, 25000),
    "25000-100000": (25000, 100000),
    "100000-500000": (100000, 500000),
    "500000-2500000": (500000, 2500000),
    "2500000-10000000": (2500000, 10000000),
    "10000000-100000000": (10000000, 100000000)}

acct_bal_prob = {"0-5000": 0.15,
                 "5000-25000": 0.30,
                 "25000-100000": 0.29,
                 "100000-500000": 0.15,
                 "500000-2500000": 0.07,
                 "2500000-10000000":0.03,
                 "10000000-100000000":0.01}

def sample_account_balance():
    # Step 1: choose bucket
    bucket = np.random.choice(
        list(acct_bal_prob.keys()),
        p=list(acct_bal_prob.values())
    )
    low, high = acct_bal_range[bucket]

    log_low, log_high = np.log(low+1), np.log(high+1)
    balance = int(np.exp(np.random.uniform(log_low, log_high)))

    return balance
#----------------------------------------------------------------------------------
# Product id distribution
product_probs = product_master_df["Percent"] / product_master_df["Percent"].sum()

def sample_product_id():
    # choose a row index based on Percent distribution
    idx = np.random.choice(product_master_df.index, p=product_probs.values)

    # return both product_id and subtype if needed
    row = product_master_df.loc[idx]
    return row["ProdSubType"]

prod_subtype = sample_product_id()
#---------------------------------------------------------------------------------
#Account start date
start_date = pd.to_datetime("2015-01-01")
end_date   = pd.to_datetime("2025-09-15")

def sample_start_date_biased():
    days_range = (end_date - start_date).days
    # exponential distribution: skew toward smaller numbers
    rand_days = int(np.random.exponential(scale=days_range/3))
    # clip to range
    rand_days = min(rand_days, days_range)
    return start_date + pd.Timedelta(days=rand_days)
#-----------------------------------------------------------------------------------
#Account End date

# Define end-date rules (in years/months)
end_date_rules = {
    # Deposits
    "Deposits - Fixed Deposit (FD)":       (6, 60),   # 6m to 5y
    "Deposits - Recurring Deposit (RD)":   (6, 60),

    # Loans
    "Loans - Personal Loan":               (12, 24),  # 1–2y
    "Loans -Home Loan":                   (60, 240), # 5–20y, can add if needed
    "Loans -Auto Loan":                   (36, 48),  # 3–4y
    "Loans -Education Loan":              (36, 84),  # 3–7y typical
    "Loans -Gold Loan":                   (24, 36),  # 2–3y
    "Loans -Two-wheeler Loan":            (24, 36),
    "Loans -Loan Against Property (LAP)": (60, 84),  # 5–7y

    # Cards
    "Cards&Payments - Credit Card – Classic":          (12, 24),
    "Cards&Payments - Credit Card – Premium / Co-branded": (12, 24),
    "Cards&Payments - Credit Card – Super Premium (Infinia/Diners Black)":    (12, 24),
    "Cards&Payments - Prepaid Card / Forex Card":      (12, 24),
    "Cards&Payments - Debit Card (linked to SA/CA)":   (60, 120), # Optional – can expire/reissue

    # Forex/NRI
    "Forex&NRI- Forex Card (Travel)":     (6, 36)   # short duration, 0.5–3y
}


def get_account_end_date(prod_subtype, start_date):
    if prod_subtype not in end_date_rules:
        return None   # For products without end dates

    min_m, max_m = end_date_rules[prod_subtype]
    duration_m = random.randint(min_m, max_m)

    end_date = start_date + relativedelta(months=duration_m)

    return end_date
#----------------------------------------------------------------------------------
# Active flag
today = pd.to_datetime("today")

def get_account_active(acct_start, acct_end, churn_prob=0.1):
    # Case 1: no end date → always active
    if acct_end is None:
        return 1

    # Case 2: natural expiry
    if acct_end < today:
        return 0
    elif acct_end >= today:
        # Add some churn probability
        return 0 if np.random.rand() < churn_prob else 1
#---------------------------------------------------------------------------------
# Average Balance

prod_map = product_master_df.set_index("ProdID")["ProdSubType"].to_dict()

def get_account_avg_balance(prod_subtype, acct_balance):
    # Deposits: average balance ~ principal
    if "Deposits - Fixed Deposit (FD)" in prod_subtype or "Deposits - Recurring Deposit (RD)" in prod_subtype:
        return acct_balance

    # Savings account: avg bal 70-90% of current balance
    if "Deposits - Savings Account" or "Deposits - Salary Account (special SA variant)" in prod_subtype:
        factor = np.random.uniform(0.7, 0.9)
        return round(acct_balance * factor)

    # Current account: avg bal 50-80% of current balance
    if "Deposits - Current Account" in prod_subtype:
        factor = np.random.uniform(0.5, 0.8)
        return round(acct_balance * factor)
    return None

    #account_df.apply(lambda row: get_account_avg_balance(row["ProdSubType"], row["acct_balance"]), axis=1)
#--------------------------------------------------------------------------------
# Account Tenure

# Assuming account_df has AcctStartDate and AcctEndDate in datetime format
today = pd.to_datetime("today")  # or pd.to_datetime("today")
placeholder_days = 1825  # 5 years

def calculate_tenure(start_date, end_date):
    if pd.notnull(end_date):  # closed / matured account
        return (end_date - start_date).days // 30
    else:  # still active
        return (placeholder_days) // 30

#---------------------------------------------------------------------------------
# Acct OD usage

def get_od_usage(prod_subtype, acct_avg_bal):
    # Only Current Accounts are eligible
    if "Deposits - Current Account" not in prod_subtype:
        return None  # Not applicable

    # Assume OD limit is ~2x average balance
    od_limit = acct_avg_bal * 2 if acct_avg_bal else 50000  # fallback

    # Assign usage band probabilistically
    rand = np.random.rand()
    if rand < 0.70:
        usage_pct = 0
    elif rand < 0.90:
        usage_pct = np.random.uniform(0.01, 0.25)
    elif rand < 0.98:
        usage_pct = np.random.uniform(0.25, 0.75)
    else:
        usage_pct = np.random.uniform(0.75, 1.0)

    return round(od_limit * usage_pct)

  #-------------------------------------------------------------------------------
  # Deqlinquent account

def get_account_delinquent(prod_subtype):
    if any(x in prod_subtype for x in ["Deposits - Fixed Deposit (FD)","Deposits - Recurring Deposit (RD)"]):
        return 0
    elif any(x in prod_subtype for x in ["Loans - Personal Loan","Loans -Home Loan","Loans -Auto Loan","Loans -Education Loan",
                                         "Loans -Gold Loan","Loans -Two-wheeler Loan","Loans -Loan Against Property (LAP)"]):
        return 1 if np.random.rand() < 0.07 else 0  # ~7% default
    elif any(x in prod_subtype for x in ["Cards&Payments - Credit Card – Classic","Cards&Payments - Credit Card – Premium / Co-branded",
                                          "Cards&Payments - Credit Card – Super Premium (Infinia/Diners Black)"]):
        return 1 if np.random.rand() < 0.05 else 0  # ~5% default
    elif "Deposits - Current Account" in prod_subtype or "Deposits - Savings Account" in prod_subtype or "Deposits - Salary Account (special SA variant)" in prod_subtype:
        return 1 if np.random.rand() < 0.015 else 0  # ~1.5% default
    else:
        return 0

In [ ]:
product_master_df["ProdSubType"] = product_master_df["ProdSubType"].str.strip()

In [ ]:
#Generate 200K accounts
num_accounts = 200000
accounts = []
acct_id_counter = 1

for cust_id in df_customers["CustomerID"]:
    num_cust_accounts = np.random.choice([1,2,3], p=[0.1,0.7,0.2])  # each customer has 1–3 accounts
    for i in range(num_cust_accounts):
        prod_subtype = sample_product_id()
        acct_start = sample_start_date_biased()
        acct_end   = get_account_end_date(prod_subtype, acct_start)
        acct_active = get_account_active(acct_start, acct_end)
        acct_balance = sample_account_balance()
        acct_avg_bal = get_account_avg_balance(prod_subtype, acct_balance)
        acct_tenure = calculate_tenure(acct_start, acct_end)
        acct_od = get_od_usage(prod_subtype, acct_avg_bal)
        acct_delinquent = get_account_delinquent(prod_subtype)
        acc = {
          "AccountID": f"ACC{acct_id_counter:06}",
          "CustomerID": cust_id,
          "Product": prod_subtype,
          "AccountStartDate": acct_start,
          "AccountEndDate":acct_end,
          "AccountBalance": acct_balance,
          "AccountAvgBal":acct_avg_bal,
          "AccountODUsage":acct_od,
          "AccountTenure":acct_tenure,
          "AccountActive": acct_active,
          "AccountDelinq":acct_delinquent
        }
        accounts.append(acc)
        acct_id_counter += 1

df_accounts = pd.DataFrame(accounts)



In [ ]:
df_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209684 entries, 0 to 209683
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   AccountID         209684 non-null  object        
 1   CustomerID        209684 non-null  object        
 2   Product           209684 non-null  object        
 3   AccountStartDate  209684 non-null  datetime64[ns]
 4   AccountEndDate    96755 non-null   datetime64[ns]
 5   AccountBalance    209684 non-null  int64         
 6   AccountAvgBal     209684 non-null  int64         
 7   AccountODUsage    5251 non-null    float64       
 8   AccountTenure     209684 non-null  int64         
 9   AccountActive     209684 non-null  int64         
 10  AccountDelinq     209684 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 17.6+ MB


In [ ]:
df_accounts.head()

,AccountID,CustomerID,Product,AccountStartDate,AccountEndDate,AccountBalance,AccountAvgBal,AccountODUsage,AccountTenure,AccountActive,AccountDelinq
0,ACC000001,CUST000001,Others - PPF (Public Provident Fund),2020-10-24,NaT,63260,46987,NaN,60,1,0
1,ACC000002,CUST000001,Loans -Auto Loan,2021-10-07,2025-05-07,36602,28073,NaN,43,0,0
2,ACC000003,CUST000002,Loans -Two-wheeler Loan,2019-08-04,2022-03-04,1124,900,NaN,31,0,0
3,ACC000004,CUST000002,Investments - Mutual Funds – Debt/Hybrid,2017-09-24,NaT,21971,17294,NaN,60,1,0
4,ACC000005,CUST000003,Cards&Payments - Credit Card – Premium / Co-br...,2015-11-06,2017-06-06,45771,35133,NaN,19,0,0


In [ ]:
#Check the accounts generated for a specific customer (to be used later in recommendations and validate recommendations)
filtered = df_accounts[df_accounts["CustomerID"] =="CUST000359"]
filtered

,AccountID,CustomerID,Product,AccountStartDate,AccountEndDate,AccountBalance,AccountAvgBal,AccountODUsage,AccountTenure,AccountActive,AccountDelinq
754,ACC000755,CUST000359,Deposits - Savings Account,2018-03-13,NaT,1902122,1707507,NaN,60,1,0
755,ACC000756,CUST000359,Forex&NRI- Forex Card (Travel),2017-11-28,2018-09-28,17688,14682,NaN,10,0,0


In [ ]:
#Check how many unique accounts are generated
df_accounts["AccountID"].nunique()

209684

In [ ]:
df_accounts["Product"].unique()

array(['Deposits - Savings Account', 'Loans - Personal Loan',
       'Others - Gold Monetization Scheme',
       'Cards&Payments - Credit Card – Premium / Co-branded',
       'Loans -Two-wheeler Loan', 'Deposits - Current Account',
       'Cards&Payments - UPI Wallet / Digital Payments',
       'Insurance - Term Insurance', 'Deposits - Fixed Deposit (FD)',
       'Others - Sovereign Gold Bonds (SGBs)',
       'Insurance - Life Insurance (ULIP / Endowment)',
       'Others - PPF (Public Provident Fund)',
       'Others - Senior Citizens’ Savings Scheme (SCSS)',
       'Cards&Payments - Debit Card (linked to SA/CA)',
       'Forex&NRI- Foreign Currency Savings/Deposits (FCNR/NRE/NRO)',
       'Loans -Education Loan',
       'Insurance - Motor Insurance (Car/Two-wheeler)',
       'Investments - SIPs (Systematic Plans)',
       'Others - Sukanya Samriddhi Yojana (SSY)',
       'Cards&Payments - Credit Card – Classic',
       'Investments - Structured Products',
       'Forex&NRI- Remittanc

In [ ]:
df_accounts["AccountStartDate"].unique(), df_accounts["AccountEndDate"].unique()

(<DatetimeArray>
 ['2021-04-11 00:00:00', '2015-08-23 00:00:00', '2015-08-04 00:00:00',
  '2016-10-10 00:00:00', '2016-01-27 00:00:00', '2015-04-30 00:00:00',
  '2015-06-07 00:00:00', '2019-04-10 00:00:00', '2015-05-05 00:00:00',
  '2015-01-30 00:00:00',
  ...
  '2024-02-23 00:00:00', '2024-07-20 00:00:00', '2019-11-14 00:00:00',
  '2023-02-14 00:00:00', '2024-11-29 00:00:00', '2024-03-14 00:00:00',
  '2023-08-28 00:00:00', '2023-10-30 00:00:00', '2023-06-06 00:00:00',
  '2023-09-07 00:00:00']
 Length: 3516, dtype: datetime64[ns],
 <DatetimeArray>
 ['2022-12-11 00:00:00', '2018-06-23 00:00:00',                 'NaT',
  '2021-07-27 00:00:00', '2017-01-07 00:00:00', '2022-07-10 00:00:00',
  '2021-06-30 00:00:00', '2017-08-23 00:00:00', '2020-01-10 00:00:00',
  '2016-11-23 00:00:00',
  ...
  '2031-11-02 00:00:00', '2030-02-27 00:00:00', '2035-05-07 00:00:00',
  '2028-03-02 00:00:00', '2035-10-04 00:00:00', '2018-07-30 00:00:00',
  '2017-02-01 00:00:00', '2020-10-02 00:00:00', '2022-04-29 

In [ ]:
df_accounts["AccountBalance"].unique()

array([   17591,     8547,    14822,    44002,   179099,       51,
          15884,   149700,   331780,      430,        3,       19,
        1626400,   109277,    97539,    42416,     8362,   161842,
         175209,    34332,    31468,    68006,    84550,     6663,
         828516,      464,     9850,   241672,   180112,    82504,
           2754,      217,    44141,    38727,   527161,    70478,
          96056,  4474130,     7055,      171,    24708,   122362,
          34755,    58947,     6629,        7,    27826,    18100,
         101924,     9754,        2,    28536,  2270387,     9552,
        2212183,    35093,    46899,    54074,    34916,    41456,
             28,    13613,    63561,    23649,    50242,   183061,
         282622,       34,    15913,    53283,    11122,     6336,
          83244,   247552,   370937,   492664,     8250,  4532249,
          80543,    13348,    37191,     5721,   113076,      631,
            211,    39008,   306980,    17019,    14087,   168

In [ ]:
df_accounts["AccountAvgBal"].unique()

array([   15130,  1173752,   785747,     1226,   554760,   653620,
         205895,      701,        1,      590,    62919,      208,
          31302,    76233,     8450,    73813,     7933,    27467,
          48509,     7378,     4738,      155,     7570,       45,
           6115,   158842,    14853,    20624,   102905,   683743,
          10072,    43778,   200670,    12470,     7008,    37694,
           8844,    55875,    38134,     5949,    62399,  5568132,
           6067,    15144,    78357,    10038,     7023,     9366,
        3304751,    22637,      403,     4823,    17538,    21872,
        1871118,    35311,        4,   207725,    83089,    37804,
          29203,    19399,     5385,     9343,    12640,    16693,
           2056,   197461,    60733,     1083,     4662,    48187,
              5,      125,       27,    59630,    13057,    14049,
          61209,     4579,   296979, 15057285,    15731,  9417479,
          38175,    57591,     5290,   183467,     7491,    13

In [ ]:
df_accounts["AccountODUsage"].unique()

array([   nan,     0.,  2874., ..., 15615.,  3356., 16155.])

In [ ]:
df_accounts["AccountTenure"].unique()

array([ 60,  14,  35, 153, 105,  13,  15,  91,  31, 164,  99,  12,  23,
        19,  34,  21, 120,  30,  50,  24,  18,  48,  98,  25, 185, 156,
        67,  89,  16,  17,  45,  10,  90, 107, 197,   8,  78,  39, 136,
       116,  49, 184,  22, 200,  43,  68,  33,  27,  88,  79,  28,  77,
         7,  20,  71, 142,  94,  29,  26,  41,  32, 194,   6,  82,  54,
       119,  59,  44,  75,  53, 224, 106, 241, 109,  42,  61, 170, 215,
        63, 114, 188,  36, 183,  46, 108, 131,  40, 174,  84,  37,  80,
        74,  11,  81,  62,  69,  57,  55,  38, 211, 123,   9, 169, 102,
        58,  66,  95, 104, 205,  51, 103, 166,  47,  56, 232,  65, 160,
       118,  64,  87,  85, 178])

In [ ]:
df_accounts["AccountActive"].unique()

array([1, 0])

In [ ]:
df_accounts["AccountDelinq"].unique()

array([0, 1])

# Interactions

In [ ]:
#Generate synthetic interactions
# Define the product type mix needed in the overall volume of accounts
product_probs = {'Deposits - Savings Account': 0.12,
 'Deposits - Current Account': 0.02,
 'Deposits - Fixed Deposit (FD)': 0.05,
 'Deposits - Recurring Deposit (RD)': 0.05,
 'Deposits - Salary Account (special SA variant)': 0.08,
 'Loans - Personal Loan': 0.06,
 'Loans -Home Loan': 0.05,
 'Loans -Auto Loan': 0.03,
 'Loans -Education Loan': 0.02,
 'Loans -Gold Loan': 0.03,
 'Loans -Two-wheeler Loan': 0.02,
 'Loans -Loan Against Property (LAP)': 0.01,
 'Cards&Payments - Credit Card – Classic': 0.04,
 'Cards&Payments - Credit Card – Premium / Co-branded': 0.02,
 'Cards&Payments - Credit Card – Super Premium (Infinia/Diners Black)': 0.01,
 'Cards&Payments - Prepaid Card / Forex Card': 0.01,
 'Cards&Payments - Debit Card (linked to SA/CA)': 0.05,
 'Cards&Payments - UPI Wallet / Digital Payments': 0.05,
 'Insurance - Life Insurance (ULIP / Endowment)': 0.01,
 'Insurance - Term Insurance': 0.01,
 'Insurance - Health Insurance': 0.01,
 'Insurance - Motor Insurance (Car/Two-wheeler)': 0.02,
 'Insurance - Travel Insurance': 0.01,
 'Insurance - Other (Accident / Critical Illness)': 0.01,
 'Investments - Mutual Funds – Equity': 0.02,
 'Investments - Mutual Funds – Debt/Hybrid': 0.02,
 'Investments - Demat + Broking Services': 0.01,
 'Investments - Bonds (Corporate/Govt)': 0.01,
 'Investments - Structured Products': 0.01,
 'Investments - Portfolio Mgmt Services (PMS/AIF)': 0.01,
 'Investments - SIPs (Systematic Plans)': 0.02,
 'Forex&NRI- Forex Card (Travel)': 0.01,
 'Forex&NRI- Foreign Currency Savings/Deposits (FCNR/NRE/NRO)': 0.01,
 'Forex&NRI- Remittances (Inward/Outward)': 0.01,
 'Forex&NRI- NRI Wealth/Advisory Services': 0.01,
 'Others - PPF (Public Provident Fund)': 0.01,
 'Others - Sukanya Samriddhi Yojana (SSY)': 0.01,
 'Others - Atal Pension Yojana (APY)': 0.01,
 'Others - Senior Citizens’ Savings Scheme (SCSS)': 0.01,
 'Others - Sovereign Gold Bonds (SGBs)': 0.01,
 'Others - Gold Monetization Scheme': 0.01,
 'Others - Other Govt Schemes / Small Savings': 0.01,
  'GEN_SUPPORT': 0.00}


In [ ]:
#Use the weights defined above in a variable
product_list = list(product_probs.keys())
product_weights = list(product_probs.values())

In [ ]:
#Define proportion of support queries, cold start interactions in the overall mix
support_prob = 0.15       # service-only
cold_start_prob = 0.08    # customer having interactions on a product but no account with that product
normal_prob = 1 - support_prob - cold_start_prob

In [ ]:
#Generate 300K interactions - part 1
num_interactions = 300_000

# --- Interaction types ---
interaction_types = [
    "View","Apply","Purchase","Decline","Complain",
    "Enquire","Pre-approved","Offer viewed","Abandon/Drop-off",
    "Upgrade/Renewal","Closure","Support"
]
interaction_probs = [0.25,0.15,0.20,0.05,0.03,0.07,0.05,0.05,0.03,0.04,0.02,0.06]

inter_types = np.random.choice(interaction_types, size=num_interactions, p=interaction_probs)

# --- Channels ---
interaction_channels = ["Mobile","Netbanking","Branch","ATM","CallCentre","RelnMgr"]
channel_probs = [0.35,0.25,0.15,0.05,0.10,0.10]

inter_channels = np.random.choice(interaction_channels, size=num_interactions, p=channel_probs)

# --- Frequencies ---
freq_values = [15, 30, 60, 90, 120]
freq_probs  = [0.4, 0.25, 0.15, 0.10, 0.10]

inter_freqs = np.random.choice(freq_values, size=num_interactions, p=freq_probs)

# --- Amount buckets ---
amount_buckets = {
    "10K and below": (0, 10000),
    "10K to 25K": (10000, 25000),
    "25K to 50K": (25000, 50000),
    "50K to 100K": (50000, 100000),
    "above 100K": (100000, 500000)
}
bucket_names = list(amount_buckets.keys())
bucket_probs = [0.50,0.20,0.15,0.10,0.05]

chosen_buckets = np.random.choice(bucket_names, size=num_interactions, p=bucket_probs)

# For each bucket, generate values
amt_values = np.zeros(num_interactions, dtype=int)
for bucket, (low, high) in amount_buckets.items():
    mask = (chosen_buckets == bucket)
    amt_values[mask] = np.random.uniform(low, high, mask.sum()).round().astype(int)

# --- Times ---
start_date = datetime(2024,9,16)
end_date   = datetime(2025,9,15)
delta_days = (end_date - start_date).days

rand_days    = np.random.randint(0, delta_days, size=num_interactions)
rand_seconds = np.random.randint(0, 24*3600, size=num_interactions)
inter_times  = [start_date + timedelta(days=int(d), seconds=int(s))
                for d, s in zip(rand_days, rand_seconds)]

In [ ]:
#Generate 300K interactions - part 2
cust_ids = np.random.choice(df_customers["CustomerID"], size=num_interactions)

# Sample productIDs with weights
prod_ids = np.random.choice(product_list, size=num_interactions, p=product_weights)

# Vectorized join attempt: map each cust_id to their available products/accounts
cust_to_products = df_accounts.groupby("CustomerID")["Product"].apply(list).to_dict()
cust_to_accounts = df_accounts.groupby(["CustomerID","Product"])["AccountID"].apply(list).to_dict()

# Build arrays
acct_ids = []
for c, p in zip(cust_ids, prod_ids):
    products = cust_to_products.get(c, [])
    if products:
        chosen_p = np.random.choice(products)
    else:
        chosen_p = np.random.choice(product_list, p=product_weights)
    prod_id = chosen_p
    accounts = cust_to_accounts.get((c, prod_id), [])
    if accounts:
        acct_id = np.random.choice(accounts)
    else:
        acct_id = "NA"
    acct_ids.append(acct_id)


In [ ]:
df_interactions = pd.DataFrame({
    "InterID": [f"INTER{i+1:06}" for i in range(num_interactions)],
    "CustomerID": cust_ids,
    "ProductID": prod_ids,
    "AccountID": acct_ids,
    "InterType": inter_types,
    "InterChannel": inter_channels,
    "InterFreq": inter_freqs,
    "InterAmtBucket": chosen_buckets,
    "InterAmt": amt_values,
    "InterTime": inter_times
})


In [ ]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   InterID         300000 non-null  object        
 1   CustomerID      300000 non-null  object        
 2   ProductID       300000 non-null  object        
 3   AccountID       300000 non-null  object        
 4   InterType       300000 non-null  object        
 5   InterChannel    300000 non-null  object        
 6   InterFreq       300000 non-null  int64         
 7   InterAmtBucket  300000 non-null  object        
 8   InterAmt        300000 non-null  int64         
 9   InterTime       300000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 22.9+ MB


In [ ]:
#Check the interactions generated for a specific customer (to be used later in recommendations and validate recommendations)
filtered = df_interactions[df_interactions["CustomerID"] =="CUST000359"]
filtered

,InterID,CustomerID,ProductID,AccountID,InterType,InterChannel,InterFreq,InterAmtBucket,InterAmt,InterTime
73966,INTER073967,CUST000359,Loans -Home Loan,ACC000755,Support,Netbanking,15,50K to 100K,67572,2025-01-30 06:06:08
113018,INTER113019,CUST000359,Investments - Mutual Funds – Debt/Hybrid,ACC000755,Purchase,Netbanking,15,25K to 50K,25910,2025-05-21 01:49:10
175694,INTER175695,CUST000359,Cards&Payments - Debit Card (linked to SA/CA),ACC000755,Purchase,RelnMgr,15,10K and below,1979,2025-04-30 07:03:29


In [ ]:
df_interactions.head()

,InterID,CustomerID,ProductID,AccountID,InterType,InterChannel,InterFreq,InterAmtBucket,InterAmt,InterTime
0,INTER000001,CUST088517,Insurance - Motor Insurance (Car/Two-wheeler),ACC185581,Purchase,CallCentre,90,10K to 25K,11646,2024-11-26 09:56:57
1,INTER000002,CUST002047,Deposits - Recurring Deposit (RD),ACC004258,Purchase,CallCentre,90,10K and below,9618,2025-08-20 21:51:37
2,INTER000003,CUST014905,Deposits - Savings Account,ACC031169,Apply,CallCentre,15,10K to 25K,13561,2025-02-05 07:17:32
3,INTER000004,CUST021429,Insurance - Health Insurance,ACC044800,View,Netbanking,15,10K to 25K,11313,2025-02-12 20:06:15
4,INTER000005,CUST042743,Cards&Payments - UPI Wallet / Digital Payments,ACC089552,Purchase,Branch,90,25K to 50K,29978,2025-04-27 14:55:08


In [ ]:
#Introduce a numeric column using 'Interaction time' column
df_interactions["InterTimeNumeric"] = (
    pd.to_datetime(df_interactions["InterTime"]) - pd.to_datetime(df_interactions["InterTime"]).min()
).dt.days

In [ ]:
#Check the number of unique accounts used in the interactions data
df_interactions["AccountID"].nunique()  # returns 3


155978

# Heterogenous data preparation

In [ ]:
#Define the customer, account and product nodes
# Customer nodes
customer_ids = df_customers["CustomerID"].tolist()
cust_id_map = {cid: i for i, cid in enumerate(customer_ids)}

# Account nodes
account_ids = df_accounts["AccountID"].tolist()
acct_id_map = {aid: i for i, aid in enumerate(account_ids)}

# Product nodes
product_ids = product_master_df["ProdSubType"].tolist()
prod_id_map = {pid: i for i, pid in enumerate(product_ids)}

In [ ]:
#Prepare Customer node feature matrix
#Categorical features → map to integer indices (for embedding layers in the model).

from sklearn.preprocessing import LabelEncoder

# Example categorical features
cat_cols = ["CustomerSegment", "CustomerCity", "CustomerRisk", "CustomerEmployment",
            "CustomerGender", "CustomerMaritalStatus", "CustomerEducation"]

df_customers_enc = df_customers.copy()
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df_customers_enc[col] = le.fit_transform(df_customers[col])
    encoders[col] = le

# Numeric features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_customers_enc[["CustomerAge", "CustomerIncome", "CustomerCreditScore"]] = scaler.fit_transform(
    df_customers_enc[["CustomerAge", "CustomerIncome", "CustomerCreditScore"]].astype(float)
)

num_cols = ["CustomerAge", "CustomerIncome", "CustomerCreditScore"]

import torch

cust_features = torch.tensor(
    df_customers_enc[num_cols + cat_cols].values, dtype=torch.float
)  # shape: [num_customers, feature_dim]

In [ ]:
#Prepare Product node feature matrix

# Example categorical features
cat_cols = ["ProdType", "ProdSubType","ProdRisk"]

df_products_enc = product_master_df.copy()
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df_products_enc[col] = le.fit_transform(product_master_df[col])
    encoders[col] = le

# Numeric features


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_products_enc[["ProdMinBalNumeric", "ProdFeeNumeric", "ProdTenorNumeric"]] = scaler.fit_transform(
    df_products_enc[["ProdMinBalNumeric", "ProdFeeNumeric", "ProdTenorNumeric"]].astype(float)
)

num_cols = ["ProdMinBalNumeric", "ProdFeeNumeric", "ProdTenorNumeric"]

import torch

prod_features = torch.tensor(
    df_products_enc[num_cols + cat_cols].values, dtype=torch.float
)  # shape: [num_products, feature_dim]

In [ ]:
#Prepare Account node feature matrix

# Example categorical features
cat_cols = ["Product"]

df_accounts_enc = df_accounts.copy()
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df_accounts_enc[col] = le.fit_transform(df_accounts[col])
    encoders[col] = le

# Numeric features

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_accounts_enc[["AccountBalance", "AccountAvgBal", "AccountODUsage","AccountTenure","AccountActive","AccountDelinq"]] = scaler.fit_transform(
    df_accounts_enc[["AccountBalance", "AccountAvgBal", "AccountODUsage","AccountTenure","AccountActive","AccountDelinq"]].astype(float)
)

num_cols = ["AccountBalance", "AccountAvgBal", "AccountODUsage","AccountTenure","AccountActive","AccountDelinq"]

import torch

acct_features = torch.tensor(
    df_accounts_enc[num_cols + cat_cols].values, dtype=torch.float
)  # shape: [num_products, feature_dim]

In [ ]:
# Prepare edge index and edge features
# Customer → Product edges
src_nodes = df_interactions["CustomerID"].map(cust_id_map).to_list()
dst_prod_nodes = df_interactions["ProductID"].map(prod_id_map).to_list()
edge_prod_index = torch.tensor([src_nodes, dst_prod_nodes], dtype=torch.long)  # shape: [2, num_edges]

# Encode categorical edge features
inter_cat_cols = ["InterType", "InterChannel", "InterAmtBucket"]
df_inter_enc = df_interactions.copy()
edge_encoders = {}
for col in inter_cat_cols:
    le = LabelEncoder()
    df_inter_enc[col] = le.fit_transform(df_interactions[col])
    edge_encoders[col] = le


scaler = MinMaxScaler()
df_inter_enc[["InterFreq", "InterAmt","InterTimeNumeric"]] = scaler.fit_transform(
    df_inter_enc[["InterFreq", "InterAmt","InterTimeNumeric"]]).astype(float)

edge_num_cols = ["InterFreq", "InterAmt","InterTimeNumeric"]

custtoprod_edge_features = torch.tensor(df_inter_enc[inter_cat_cols + edge_num_cols].values, dtype=torch.float)


In [ ]:
# Customer → Account edges
df_valid_acct = df_interactions[df_interactions["AccountID"] != "NA"].copy()
df_valid_acct["dst"] = df_valid_acct["AccountID"].map(acct_id_map)
df_valid_acct["src"] = df_valid_acct["CustomerID"].map(cust_id_map)
df_valid_acct = df_valid_acct.dropna(subset=["src","dst"])

src_nodes = df_valid_acct["src"].astype(int).to_list()
dst_acct_nodes = df_valid_acct["dst"].astype(int).to_list()

# src_nodes = df_interactions["CustomerID"].map(cust_id_map).to_list()
# dst_acct_nodes = df_interactions["AccountID"].map(prod_id_map).to_list()
edge_acct_index = torch.tensor([src_nodes, dst_acct_nodes], dtype=torch.long)  # shape: [2, num_edges]

custtoacct_edge_features = torch.tensor(df_inter_enc[inter_cat_cols + edge_num_cols].values, dtype=torch.float)

In [ ]:
#Build PyG heterogeneous graph with the nodes and edges defined above
from torch_geometric.data import HeteroData

data = HeteroData()

# Nodes
data["customer"].x = cust_features
data["product"].x = prod_features
data["account"].x = acct_features

# Edges
data["customer", "interacts", "product"].edge_index = edge_prod_index
data["customer", "interacts", "product"].edge_attr = custtoprod_edge_features
data["customer", "interacts", "account"].edge_index = edge_acct_index
data["customer", "interacts", "account"].edge_attr = custtoacct_edge_features


# Model Build and Train

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.loader import HGTLoader
import math
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
from collections import defaultdict
import random

# Define HGT Transformer layer
class HGTLayer(nn.Module):
    """
    Heterogeneous Graph Transformer Layer
    Implements the core HGT attention mechanism with type-specific transformations
    """
    def __init__(self, node_types, edge_types, d_model, num_heads=8, dropout=0.1):
        super(HGTLayer, self).__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Type-specific linear transformations for attention
        self.k_linears = nn.ModuleDict()
        self.q_linears = nn.ModuleDict()
        self.v_linears = nn.ModuleDict()
        self.a_linears = nn.ModuleDict()  # Attention edge type embeddings

        # Message and attention transformations for each edge type
        for edge_type in edge_types:
            src_type, relation, dst_type = edge_type
            edge_key = f"{src_type}_{relation}_{dst_type}"

            # Source node transformations (Keys and Values)
            self.k_linears[edge_key] = nn.Linear(d_model, d_model)
            self.v_linears[edge_key] = nn.Linear(d_model, d_model)

            # Target node transformations (Queries)
            self.q_linears[edge_key] = nn.Linear(d_model, d_model)

            # Edge type attention weights
            self.a_linears[edge_key] = nn.Linear(d_model, num_heads)

        # Skip connections and layer norm for each node type
        self.skip_linears = nn.ModuleDict()
        self.norms = nn.ModuleDict()

        for node_type in node_types:
            self.skip_linears[node_type] = nn.Linear(d_model, d_model)
            self.norms[node_type] = nn.LayerNorm(d_model)

        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, x_dict, edge_index_dict, edge_attr_dict=None):
        """
        Forward pass of HGT layer

        Args:
            x_dict: Dict of node embeddings {node_type: tensor}
            edge_index_dict: Dict of edge indices {edge_type: tensor}
            edge_attr_dict: Dict of edge features {edge_type: tensor} (optional)
        """

        # Store original embeddings for skip connections
        x_dict_orig = {k: v.clone() for k, v in x_dict.items()}

        # Collect messages for each target node type
        messages = defaultdict(list)

        for edge_type, edge_index in edge_index_dict.items():
            if edge_index.size(1) == 0:  # Skip empty edge types
                continue

            src_type, relation, dst_type = edge_type
            edge_key = f"{src_type}_{relation}_{dst_type}"

            if edge_key not in self.k_linears:
                continue

            # Get source and target node embeddings
            src_x = x_dict[src_type]
            dst_x = x_dict[dst_type]

            # Source and target node indices
            src_idx, dst_idx = edge_index[0], edge_index[1]

            # Get source embeddings for this edge
            src_emb = src_x[src_idx]  # [num_edges, d_model]
            dst_emb = dst_x[dst_idx]  # [num_edges, d_model]

            # Apply type-specific transformations
            K = self.k_linears[edge_key](src_emb)  # [num_edges, d_model]
            V = self.v_linears[edge_key](src_emb)  # [num_edges, d_model]
            Q = self.q_linears[edge_key](dst_emb)  # [num_edges, d_model]

            # Reshape for multi-head attention
            batch_size = K.size(0)
            K = K.view(batch_size, self.num_heads, self.d_k)  # [num_edges, num_heads, d_k]
            V = V.view(batch_size, self.num_heads, self.d_k)
            Q = Q.view(batch_size, self.num_heads, self.d_k)

            # Compute attention weights
            attention_scores = torch.sum(Q * K, dim=-1) / math.sqrt(self.d_k)  # [num_edges, num_heads]

            # Add edge type specific attention bias
            edge_attention = self.a_linears[edge_key](dst_emb)  # [num_edges, num_heads]
            attention_scores = attention_scores + edge_attention

            # Apply edge features if available
            if edge_attr_dict and edge_type in edge_attr_dict:
                edge_features = edge_attr_dict[edge_type]
                if edge_features.size(0) == attention_scores.size(0):
                    # Simple way to incorporate edge features
                    edge_weights = torch.mean(edge_features, dim=-1, keepdim=True)  # [num_edges, 1]
                    attention_scores = attention_scores * edge_weights

            attention_weights = F.softmax(attention_scores, dim=0)  # Softmax over edges to same target
            attention_weights = self.dropout_layer(attention_weights)

            # Apply attention to values
            attended_values = attention_weights.unsqueeze(-1) * V  # [num_edges, num_heads, d_k]
            attended_values = attended_values.view(batch_size, -1)  # [num_edges, d_model]

            # Aggregate messages for target nodes
            messages[dst_type].append((dst_idx, attended_values))

        # Aggregate messages and update node embeddings
        new_x_dict = {}
        for node_type in x_dict.keys():  # Only iterate over existing node types in the batch
            if node_type in messages:
                # Aggregate all messages for this node type
                all_dst_idx = []
                all_messages = []

                for dst_idx, msg in messages[node_type]:
                    all_dst_idx.append(dst_idx)
                    all_messages.append(msg)

                if all_messages:
                    # Concatenate all messages
                    all_dst_idx = torch.cat(all_dst_idx, dim=0)
                    all_messages = torch.cat(all_messages, dim=0)

                    # Aggregate messages by target node (sum aggregation)
                    num_nodes = x_dict[node_type].size(0)
                    aggregated = torch.zeros(num_nodes, self.d_model, device=x_dict[node_type].device)
                    aggregated.index_add_(0, all_dst_idx, all_messages)

                    # Apply skip connection and layer norm
                    skip_out = self.skip_linears[node_type](x_dict_orig[node_type])
                    new_x_dict[node_type] = self.norms[node_type](aggregated + skip_out)
                else:
                    # No messages received, just apply skip connection
                    new_x_dict[node_type] = self.norms[node_type](
                        self.skip_linears[node_type](x_dict_orig[node_type])
                    )
            else:
                # No messages for this node type
                new_x_dict[node_type] = self.norms[node_type](
                    self.skip_linears[node_type](x_dict_orig[node_type])
                )

        return new_x_dict

In [ ]:
# Define HGT model with embedding layer, transformer layer and prediction layer
class HGTModel(nn.Module):
    """
    Complete HGT (Heterogeneous Graph Transformer) Model
    """
    def __init__(self, node_types, edge_types, node_feat_dims, d_model=128,
                 num_layers=2, num_heads=8, dropout=0.1):
        super(HGTModel, self).__init__()

        self.d_model = d_model

        # Node type embeddings - project different feature dimensions to d_model
        self.node_embeddings = nn.ModuleDict()
        for node_type in node_types:
            input_dim = node_feat_dims[node_type]
            self.node_embeddings[node_type] = nn.Sequential(
                nn.Linear(input_dim, d_model),
                nn.ReLU(),
                nn.Dropout(dropout)
            )

        # HGT layers
        self.hgt_layers = nn.ModuleList([
            HGTLayer(node_types, edge_types, d_model, num_heads, dropout)
            for _ in range(num_layers)
        ])

        # Task-specific prediction heads
        self.prediction_heads = nn.ModuleDict()

        # Customer-Product recommendation head
        self.prediction_heads['customer_product'] = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 1),
            nn.Sigmoid()
        )

        # Customer-Account recommendation head
        self.prediction_heads['customer_account'] = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1),
            nn.Sigmoid()
        )

        # Node classification head (e.g., customer risk)
        self.prediction_heads['node_classification'] = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 3),  # 3 risk categories
            nn.Softmax(dim=-1)
        )

    def forward(self, x_dict, edge_index_dict, edge_attr_dict=None):
        """Forward pass through the HGT model"""

        # Apply node type specific embeddings
        h_dict = {}
        for node_type, features in x_dict.items():
            h_dict[node_type] = self.node_embeddings[node_type](features)

        # Apply HGT layers
        for hgt_layer in self.hgt_layers:
            h_dict = hgt_layer(h_dict, edge_index_dict, edge_attr_dict)

        return h_dict

    def predict_customer_product(self, customer_emb, product_emb):
        """Predict customer-product interaction"""
        combined = torch.cat([customer_emb, product_emb], dim=-1)
        return self.prediction_heads['customer_product'](combined)

    def predict_customer_account(self, customer_emb, account_emb):
        """Predict customer-account interaction"""
        combined = torch.cat([customer_emb, account_emb], dim=-1)
        return self.prediction_heads['customer_account'](combined)

    def predict_node_class(self, node_emb):
        """Predict node classification (e.g., risk level)"""
        return self.prediction_heads['node_classification'](node_emb)

In [ ]:
#Use the heterogenous data generated earlier
def create_sample_hetero_data():
      data['customer'].x = cust_features
      data['product'].x = prod_features
      data['account'].x = acct_features

      data['customer', 'interacts', 'product'].edge_index = edge_prod_index
      data['customer', 'interacts', 'product'].edge_attr = custtoprod_edge_features
      data['customer', 'interacts', 'account'].edge_index = edge_acct_index
      data['customer', 'interacts', 'account'].edge_attr = custtoacct_edge_features

      return data

In [ ]:
# Define the function to train the model (with 100 epochs)
def train_hgt_model(data, epochs=100, lr=0.001, device='cpu'):
    """Train the HGT model"""

    # Move data to device
    data = data.to(device)

    # Define model parameters
    node_types = list(data.node_types)
    edge_types = list(data.edge_types)

    # Get node feature dimensions
    node_feat_dims = {}
    for node_type in node_types:
        node_feat_dims[node_type] = data[node_type].x.size(1)

    print("Graph Structure:")
    print(f"Node types: {node_types}")
    print(f"Edge types: {edge_types}")
    print(f"Node feature dimensions: {node_feat_dims}")

    for node_type in node_types:
        print(f"{node_type}: {data[node_type].x.shape[0]} nodes")

    for edge_type in edge_types:
        edge_data = data[edge_type]
        print(f"{edge_type}: {edge_data.edge_index.shape[1]} edges")
        if hasattr(edge_data, 'edge_attr') and edge_data.edge_attr is not None:
            print(f"  - Edge features: {edge_data.edge_attr.shape}")

    # Initialize model
    model = HGTModel(
        node_types=node_types,
        edge_types=edge_types,
        node_feat_dims=node_feat_dims,
        d_model=128,
        num_layers=2,
        num_heads=8,
        dropout=0.1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    # Create training data for customer-product recommendations
    #edge_index = data['customer', 'interacts', 'product'].edge_index
    edge_index = edge_prod_index

    # Positive samples
    pos_edges = edge_index.T  # [num_edges, 2]
    pos_labels = torch.ones(pos_edges.size(0), device=device)

    # Negative samples
    num_neg = pos_edges.size(0)
    neg_customers = torch.randint(0, data['customer'].x.size(0), (num_neg,), device=device)
    neg_products = torch.randint(0, data['product'].x.size(0), (num_neg,), device=device)
    neg_edges = torch.stack([neg_customers, neg_products], dim=1)
    neg_labels = torch.zeros(num_neg, device=device)

    # Combine positive and negative samples
    all_edges = torch.cat([pos_edges, neg_edges], dim=0)
    all_labels = torch.cat([pos_labels, neg_labels], dim=0)

    # Shuffle the data
    perm = torch.randperm(all_edges.size(0))
    all_edges = all_edges[perm]
    all_labels = all_labels[perm]

    print(f"\nTraining data: {pos_edges.size(0)} positive, {num_neg} negative samples")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    model.train()

    print("\nTraining HGT Model...")
    print("-" * 50)

    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass
        h_dict = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)

        # Get embeddings for prediction
        customer_emb = h_dict['customer'][all_edges[:, 0]]
        product_emb = h_dict['product'][all_edges[:, 1]]

        # Predict interactions
        predictions = model.predict_customer_product(customer_emb, product_emb).squeeze()

        # Calculate loss
        loss = criterion(predictions, all_labels)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 20 == 0:
            model.eval()
            with torch.no_grad():
                h_dict_eval = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)
                customer_emb_eval = h_dict_eval['customer'][all_edges[:, 0]]
                product_emb_eval = h_dict_eval['product'][all_edges[:, 1]]
                pred_probs = model.predict_customer_product(customer_emb_eval, product_emb_eval).squeeze()

                auc = roc_auc_score(all_labels.cpu().numpy(), pred_probs.cpu().numpy())
                acc = accuracy_score(all_labels.cpu().numpy(), (pred_probs > 0.5).cpu().numpy())

            model.train()
            print(f'Epoch {epoch+1:3d}/100 | Loss: {loss:.4f} | AUC: {auc:.4f} | Acc: {acc:.4f}')

    return model, data

In [ ]:
#Define a function to generate recommendations
def generate_recommendations(model, data, customer_id, top_k=5):
    """Generate top-k product recommendations for a customer"""
    model.eval()

    with torch.no_grad():
        # Get node embeddings
        h_dict = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)

        # Get customer embedding
        cust_idx = cust_id_map[customer_id]
        customer_emb = h_dict['customer'][cust_idx:cust_idx+1]  # [1, d_model]

        # Get all product embeddings
        product_embs = h_dict['product']  # [num_products, d_model]
        num_products = product_embs.size(0)

        # Expand customer embedding to match all products
        customer_emb_expanded = customer_emb.expand(num_products, -1)  # [num_products, d_model]

        # Predict scores for all products
        scores = model.predict_customer_product(customer_emb_expanded, product_embs).squeeze()

        desired_k = 5
        overfetch_k = top_k + 5  # Fetch extra to allow for exclusions

        # Get top-k recommendations
        top_k_scores, top_k_indices = torch.topk(scores, overfetch_k)

        recommendations = []
        index_to_prodname = {v: k for k, v in prod_id_map.items()}
        excluded_product = "GEN_SUPPORT"
        rank = 1
        for i, (idx, score) in enumerate(zip(top_k_indices, top_k_scores)):
            prod_name = index_to_prodname.get(idx.item(),f"Product {idx.item()}")
            if prod_name == excluded_product:
                  continue  # Skip GEN_SUPPORT
            recommendations.append({
                'rank': rank,
                'product_id': prod_name,
                'score': score.item()
            })

            rank += 1
            if rank > desired_k:
                break  # Stop once we have 5 valid recommendations

    return recommendations

# Generate Product Recommendations

In [ ]:
#Generate recommendations for a specific customer
if __name__ == "__main__":
    # Create sample data (replace with your actual data)
    data = create_sample_hetero_data()

    # Train the model
    model, trained_data = train_hgt_model(data, epochs=100, lr=0.001)

    # Get recommendations for any customer
    customer_id = 'CUST000359'  # Change this to any customer ID
    recommendations = generate_recommendations(model, trained_data, customer_id, top_k=5)

    customer_name = df_customers.loc[df_customers['CustomerID'] == customer_id, 'CustomerName'].values[0]
    print(f"\nTop 5 Product Recommendations for {customer_name} (Customer ID: {customer_id}):")
    #print(f"\nTop 5 Product Recommendations for Customer {customer_id}:")
    for rec in recommendations:
        print(f"Rank {rec['rank']}: Product {rec['product_id']} (Score: {rec['score']:.4f})")

Graph Structure:
Node types: ['customer', 'product', 'account']
Edge types: [('customer', 'interacts', 'product'), ('customer', 'interacts', 'account')]
Node feature dimensions: {'customer': 10, 'product': 6, 'account': 7}
customer: 100000 nodes
product: 43 nodes
account: 209684 nodes
('customer', 'interacts', 'product'): 300000 edges
  - Edge features: torch.Size([300000, 6])
('customer', 'interacts', 'account'): 300000 edges
  - Edge features: torch.Size([300000, 6])

Training data: 300000 positive, 300000 negative samples
Model parameters: 388,901

Training HGT Model...
--------------------------------------------------
Epoch  20/100 | Loss: 0.6567 | AUC: 0.6762 | Acc: 0.6130
Epoch  40/100 | Loss: 0.6222 | AUC: 0.7182 | Acc: 0.6780
Epoch  60/100 | Loss: 0.6160 | AUC: 0.7204 | Acc: 0.6780
Epoch  80/100 | Loss: 0.6122 | AUC: 0.7206 | Acc: 0.6780
Epoch 100/100 | Loss: 0.6116 | AUC: 0.7206 | Acc: 0.6751

Top 5 Product Recommendations for Yahvi Kant (Customer ID: CUST000359):
Rank 1: Pro